In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Lambda, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample corpus
corpus = [
    "the cat sat on the mat",
    "the dog sat on the log",
    "cats and dogs are great pets",
    "the mat is soft and warm"
]

# Preprocess text: Tokenization and lowercasing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1  # +1 for padding

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(corpus)


2024-10-07 02:48:52.455903: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-07 02:48:52.507897: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 02:48:52.508009: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 02:48:52.508136: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 02:48:52.517996: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-07 02:48:52.518871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
## Stage b: Generate Training Data
def generate_training_data(sequences, window_size=2):
    contexts = []
    targets = []
    
    for sequence in sequences:
        for i in range(window_size, len(sequence) - window_size):
            context = sequence[i - window_size:i] + sequence[i + 1:i + window_size + 1]
            target = sequence[i]
            contexts.append(context)
            targets.append(target)
    
    return np.array(contexts), np.array(targets)

X, y = generate_training_data(sequences)

# Pad sequences for consistent input shape
X = pad_sequences(X, maxlen=4)  # Adjust maxlen based on context size


In [8]:
## Stage c: Train Model
# Define CBOW model architecture
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=10, input_length=4))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))  # Average embeddings
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100)


Epoch 1/100
1/1 [==============================] - 1s 537ms/step - loss: 2.8329 - accuracy: 0.1250
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 2.8297 - accuracy: 0.2500
Epoch 3/100
1/1 [==============================] - 0s 20ms/step - loss: 2.8266 - accuracy: 0.3750
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 2.8234 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 2.8203 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 12ms/step - loss: 2.8171 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 15ms/step - loss: 2.8140 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 16ms/step - loss: 2.8109 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 21ms/step - loss: 2.8077 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 2.8046 - accuracy: 0.6250
Epoch 11/100
1/1 [

In [16]:
## Stage d: Output
# Get word embeddings from the trained model
word_embeddings = model.layers[0].get_weights()[0]

# Create a mapping of words to their embeddings
word_index = tokenizer.word_index


print('Vocabulary Size:', len(word_index))
print('Vocabulary Sample:', list(word_index.items())[:10],"\n\n")


embeddings_dict = {word: word_embeddings[idx] for word, idx in word_index.items()}

# Output the embeddings for each word in a structured format
print("{:<10} | {}".format("Word", "Embedding"))
print("-" * 40)
for word, embedding in embeddings_dict.items():
    print("{:<10} | {}".format(word, np.round(embedding, 3)))


Vocabulary Size: 16
Vocabulary Sample: [('the', 1), ('sat', 2), ('on', 3), ('mat', 4), ('and', 5), ('cat', 6), ('dog', 7), ('log', 8), ('cats', 9), ('dogs', 10)] 


Word       | Embedding
----------------------------------------
the        | [ 0.107 -0.068  0.133 -0.125 -0.146 -0.064  0.076 -0.07   0.142  0.153]
sat        | [ 0.038 -0.151  0.118  0.161  0.06  -0.075  0.104 -0.105  0.121  0.084]
on         | [ 0.072 -0.111  0.172 -0.084 -0.104 -0.114  0.073 -0.098 -0.087  0.139]
mat        | [ 0.103  0.11  -0.086  0.096 -0.078  0.16   0.119 -0.071  0.068  0.076]
and        | [-0.143 -0.113  0.105  0.156 -0.095  0.115 -0.062 -0.159 -0.112  0.086]
cat        | [ 0.131 -0.106  0.151 -0.074 -0.084 -0.141  0.093 -0.097  0.119  0.159]
dog        | [ 0.145 -0.159  0.165 -0.136 -0.096 -0.056  0.111 -0.126  0.162  0.143]
log        | [ 0.05  -0.104  0.103  0.153  0.104 -0.148  0.153 -0.134  0.14   0.104]
cats       | [-0.148 -0.14   0.139  0.131  0.075  0.154 -0.112  0.126 -0.066  0.052]
dogs  